Importando Pacotes

In [2]:
# !pip install -r requirements.txt

In [3]:
from joblib import load
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import  MinMaxScaler

Configurações do codigo

In [4]:
pd.set_option('display.max_columns', None)
season = 2023

### Aplicação do Modelo

In [5]:
model = load(f'modelos/modelo_regressao_logistica_brasileirao_{season}.joblib')

In [6]:
def generate_df_feature_label(df:pd.DataFrame, normalize=False):
    global list_columns

    _list = ['teams.home.id', 'score.halftime.home', 'home.pf.drw', 'home.pf.win', 'home.pf.los',
             'home.BLPS', 'home.BSH', 'home.CK', 'home.EXPGL', 'home.GKSV',
             'home.OFSD', 'home.RCD', 'home.SHFG', 'home.SHIN',
             'home.SHOG', 'home.SHOU'
          ]
    list_columns = _list + [x.replace('home', 'away') for x in _list] + ['HTR']

    features = df[list_columns]
    labels =  df[['result']]

    if normalize:
        features = pd.DataFrame(MinMaxScaler().fit_transform(features), columns=features.columns)
        labels = pd.DataFrame(MinMaxScaler().fit_transform(labels), columns=labels.columns).astype('str')

    return features, labels

In [7]:
df_previsao = pd.read_excel('data/trusted/base_brasileirao_2023.xlsx')
df_previsao = df_previsao[df_previsao['league.round'] >= 35]

In [8]:
X, y = generate_df_feature_label(df_previsao, True)

In [9]:
df_previsao['previsao'] = model.predict(X)

df_previsao['previsao_time'] = \
    np.where( df_previsao['previsao'] == '1.0', df_previsao['teams.home.name'],
    np.where( df_previsao['previsao'] == '0.0', df_previsao['teams.away.name'],
    np.where( df_previsao['previsao'] == '0.5', 'Empate',
    None)))

df_previsao['halftime_score'] = df_previsao[["score.halftime.home", "score.halftime.away"]]\
                                    .astype('str')\
                                    .apply("x".join, axis=1)

df_previsao['fulltime_score'] = df_previsao[["score.fulltime.home", "score.fulltime.away"]]\
                                    .astype('str')\
                                    .apply("x".join, axis=1)

In [10]:
df_previsao = df_previsao[[
    'fixture.date', 'league.round', 'teams.home.name', 'teams.away.name',
    'previsao', 'previsao_time', 'result.team', 'halftime_score', 'fulltime_score',
    'home.pf.drw', 'home.pf.win', 'home.pf.los', 'away.pf.drw', 'away.pf.win',
    'away.pf.los', 'home.BLPS', 'home.BSH', 'home.CK', 'home.EXPGL', 'home.FOU',
    'home.GKSV', 'home.OFSD', 'home.PAS%', 'home.PASAC', 'home.RCD', 'home.SHFG',
    'home.SHIN', 'home.SHOG', 'home.SHOU', 'home.TPAS', 'home.TSH', 'home.YCD',
    'away.BLPS', 'away.BSH', 'away.CK', 'away.EXPGL', 'away.FOU',
    'away.GKSV', 'away.OFSD', 'away.PAS%', 'away.PASAC', 'away.RCD',
    'away.SHFG', 'away.SHIN', 'away.SHOG', 'away.SHOU', 'away.TPAS',
    'away.TSH', 'away.YCD'
]]

### Output Resultados

In [11]:
df_previsao.to_excel(
    f'data/refined/previsao_brasileirao_{season}/predict_{datetime.now().strftime('%y%m%d %H%M%S')}.xlsx',
    index=False
)

In [12]:
scores = model.score(X, y)
print('accuracy: {:.2f}'.format(scores.mean()))

accuracy: 0.75
